# Importando Pacotes

In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import ast
import re

from langdetect import detect, DetectorFactory
from datetime import datetime
from src import constants
from src.features import features

In [4]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [5]:
today = datetime.now()

In [6]:
DetectorFactory.seed = 0

In [7]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return "unknown"

In [8]:
def format_interval(interval_str):
    match = re.match(r"\((.*?), (.*?)\]", interval_str)
    if match:
        lower = int(round(float(match.group(1))))
        upper = int(round(float(match.group(2))))
        return f"[{lower} - {upper}]"
    else:
        return interval_str 

# Leitura das bases

In [8]:
listings = pd.read_csv("data/raw/listings.csv")
calendar = pd.read_csv("data/raw/calendar.csv")
reviews = pd.read_csv("data/raw/reviews.csv")

In [9]:
calendar = calendar\
.query(f"""date == '{listings['last_scraped'].unique()[0]}'""")\
.reset_index(drop=True)

# Contexto das bases

Todos os dados utilizados aqui foram obtidos a partir do portal [Inside Airbnb](http://insideairbnb.com/get-the-data.html).


- listings.csv: Contém informações detalhadas sobre os imóveis listados em Hong Kong, incluindo dados como tipo de propriedade, localização, preço, capacidade, e comodidades oferecidas.

- calendar.csv: Fornece informações sobre a disponibilidade e reservas de imóveis em Hong Kong, incluindo datas de check-in e check-out, status de reservas e preços por noite.

- reviews.csv: Inclui dados sobre as avaliações e comentários deixados pelos hóspedes para imóveis em Hong Kong, com detalhes como classificações, comentários dos hóspedes e datas das avaliações.

**Base excluída:**

``calendar``:

Após análise dos dados fornecidos pelo Airbnb de Hong Kong, foram observadas divergências entre os preços dos imóveis nas bases de dados listings.csv e calendar.csv.

Especificamente, foi constatado que em 56% dos imóveis, os preços registrados na base listings não correspondem aos preços na base calendar.

Dado esse cenário, foi decidido utilizar os preços presentes na base listings como a fonte principal de informação. 


# Dicionário

### Listings

| Variável  |   Tipo   |       Descrição       | referencia |
|-----------|----------|-----------------------|--|
| id	| int64	|Identificador único do anúncio| Imóvel|
|listing_url|	object	|URL do anúncio	Imóvel|	Imóvel|
|scrape_id|	int64	|Identificador único da raspagem|	Imóvel|
|last_scraped|	object	|Data da última raspagem de dados|	Imóvel|
| source|	object	|Fonte dos dados|	Imóvel|
| name|	object	|Nome do imóvel|	Imóvel|
| description|	object|	Descrição do imóvel	|Imóvel|
| neighborhood_overview|	object|	Visão geral da vizinhança|	Localização|
| picture_url|	object	|URL da imagem principal do imóvel|	Imóvel|
| host_id|	int64|	Identificador único do anfitrião|	Proprietário|
| host_url|	object|	URL do perfil do anfitrião	|Proprietário|
| host_name|	object	|Nome do anfitrião|	Proprietário|
| host_since|	object	|Data desde quando o anfitrião está ativo	|Proprietário|
| host_location|	object|	Localização do anfitrião|	Proprietário|
| host_about|	object|	Descrição sobre o anfitrião|	Proprietário|
| host_response_time|	object	|Tempo de resposta do anfitrião	|Proprietário|
| host_response_rate|	object	|Taxa de resposta do anfitrião	|Proprietário|
| host_acceptance_rate|	object	|Taxa de aceitação do anfitrião	|Proprietário|
| host_is_superhost|	object	|Indica se o anfitrião é um superhost	|Proprietário|
| host_thumbnail_url|	object	|URL da miniatura da foto do anfitrião	|Proprietário|
| host_picture_url|	object	|URL da foto do anfitrião	|Proprietário|
| host_neighbourhood|	object	|Bairro do anfitrião	|Proprietário|
| host_listings_count|	int64	|Número de imóveis do anfitrião	|Proprietário|
| host_total_listings_count|	int64	|Total de imóveis do anfitrião	|Proprietário|
| host_verifications|	object	|Verificações do anfitrião	|Proprietário|
| host_has_profile_pic|	object	|Indica se o anfitrião tem foto de perfil	|Proprietário|
| host_identity_verified|	object	|Indica se a identidade do anfitrião foi verificada	|Proprietário|
| neighbourhood|	object	|Bairro do imóvel	|Localização|
| neighbourhood_cleansed|	object	|Bairro do imóvel (limpo)	|Localização|
| neighbourhood_group_cleansed|	float64	|Grupo de bairros do imóvel (limpo)	|Localização|
| latitude|	float64	|Latitude do imóvel	|Localização|
| longitude|	float64	|Longitude do imóvel	|Localização|
| property_type|	object	|Tipo de propriedade	|Imóvel|
| room_type|	object	|Tipo de quarto	|Imóvel|
| accommodates|	int64	|Capacidade de hóspedes	|Imóvel|
| bathrooms|	float64	|Número de banheiros	|Imóvel|
| bathrooms_text|	object	|Descrição dos banheiros	|Imóvel|
| bedrooms|	float64	|Número de quartos|	Imóvel|
| beds|	float64	|Número de camas	|Imóvel|
| amenities|	object	|Comodidades do imóvel	|Imóvel|
| price|	object	|Preço por noite	|Imóvel|
| minimum_nights|	int64	|Número mínimo de noites	|Imóvel|
| maximum_nights|	int64	|Número máximo de noites	|Imóvel|
| minimum_minimum_nights|	int64	|Mínimo de noites mínimas permitido	|Imóvel|
| maximum_minimum_nights|	int64	|Máximo de noites mínimas permitido	|Imóvel|
| minimum_maximum_nights|	int64	|Mínimo de noites máximas permitido	|Imóvel|
| maximum_maximum_nights|	int64	|Máximo de noites máximas permitido	|Imóvel|
| minimum_nights_avg_ntm|	float64	|Média de noites mínimas no último mês	|Imóvel|
| maximum_nights_avg_ntm|	float64	|Média de noites máximas no último mês	|Imóvel|
| calendar_updated|	float64	|Frequência de atualização do calendário	|Imóvel|
| has_availability|	object |Indica se há disponibilidade	|Imóvel|
| availability_30|	int64	|Disponibilidade nos próximos 30 dias	|Imóvel|
| availability_60|	int64	|Disponibilidade nos próximos 60 dias	|Imóvel|
| availability_90|	int64	|Disponibilidade nos próximos 90 dias	|Imóvel|
| availability_365|	int64	|Disponibilidade nos próximos 365 dias	|Imóvel|
| calendar_last_scraped|	object	|Data da última raspagem do calendário	|Imóvel|
| number_of_reviews|	int64	|Número de avaliações	|Imóvel|
| number_of_reviews_ltm|	int64	|Número de avaliações no último ano	|Imóvel|
| number_of_reviews_l30d|	int64	|Número de avaliações nos últimos 30 dias	|Imóvel|
| first_review|	object	|Data da primeira avaliação	|Imóvel|
| last_review|	object	|Data da última avaliação	|Imóvel|
| review_scores_rating|	float64	|Nota geral das avaliações	|Imóvel|
| review_scores_accuracy|	float64	|Nota de precisão das avaliações	|Imóvel|
| review_scores_cleanliness|	float64	|Nota de limpeza das avaliações	|Imóvel|
| review_scores_checkin|	float64	|Nota de check-in das avaliações	|Imóvel|
| review_scores_communication|	float64	|Nota de comunicação das avaliações|	Imóvel|
| review_scores_location|	float64	|Nota de localização das avaliações|	Imóvel|
| review_scores_value|	float64	|Nota de valor das avaliações	|Imóvel|
| license|	float64	|Licença do imóvel	|Imóvel|
| instant_bookable|	object |Indica se o imóvel pode ser reservado instantaneamente	|Imóvel|
| calculated_host_listings_count|	int64	|Contagem de imóveis calculada do anfitrião	|Proprietário|
| calculated_host_listings_count_entire_homes|	int64	|Contagem de imóveis inteiros do anfitrião	|Proprietário|
| calculated_host_listings_count_private_rooms|	int64	|Contagem de quartos privados do anfitrião	|Proprietário|
| calculated_host_listings_count_shared_rooms|	int64	|Contagem de quartos compartilhados do anfitrião	|Proprietário|
| reviews_per_month|	float64	|Número de avaliações por mês	|Imóvel|

### Reviews

| Variável  |   Tipo   |       Descrição       | referencia |
|-----------|----------|-----------------------|--|
|listing_id|	int64|	Identificador único do anúncio associado ao comentário|	Imóvel|
|id|	int64|	Identificador único da review|	Review|
|date|	object|	Data da review|	Review|
|reviewer_id|	int64|	Identificador único do usuário que fez a review|	Usuário|
|reviewer_name|	object|	Nome do usuário que fez a review|	Usuário|
|comments|	object|	Conteúdo do comentário feito pelo usuário|	Review|


# Preparação dos dados

- Detecção de missing
- Remoção de colunas 
- Alteração de tipagem dos dados

In [9]:
listings['price'] = listings['price'].str.replace('[$,]', '', regex=True)
listings['price'] = listings['price'].astype(float)

In [10]:
listings[['host_response_rate', 'host_acceptance_rate']] = \
listings[['host_response_rate', 'host_acceptance_rate']]\
.apply(lambda x: x.str.replace('[%,]', '', regex=True))

In [11]:
listings[['host_since', 'last_review', 'first_review']] = \
listings[['host_since', 'last_review', 'first_review']]\
.apply(pd.to_datetime)

In [ ]:
reviews['language'] = reviews['comments'].apply(detect_language)

In [ ]:
reviews = reviews.query("""language == 'en'""")\
.reset_index(drop=True)

In [ ]:
reviews_aggregated = reviews.groupby('listing_id').agg({
    'comments': lambda x: ' '.join(x)
}).reset_index()\
.rename(columns={'listing_id':'id'})\
.copy()

In [ ]:
listings = pd.merge(
    listings,
    reviews_aggregated,
    how='left',
    on='id'
)

In [ ]:
listings.to_parquet('data/bronze/listings.parquet')

# Feature Enginner

In [9]:
listings = pd.read_parquet('data/bronze/listings.parquet')

In [10]:
listings['host_years_active'] = (today - listings['host_since']).dt.days // 365.25

In [11]:
binary_columns = [
    'instant_bookable', 'host_is_superhost', 
    'has_availability', 'host_identity_verified',
    'host_has_profile_pic'
]

In [13]:
listings[binary_columns] = listings[binary_columns]\
.replace({'t': 1, 'f': 0}).fillna(0) 

In [14]:
listings['price_range'] = pd\
.qcut(listings['price'], q=10)\
.astype(str)

In [15]:
listings['host_listings_count_range'] = pd\
.qcut(listings['host_total_listings_count'], q=5)\
.astype(str)

In [16]:
listings['host_years_active_range'] = pd\
.qcut(listings['host_years_active'], q=4)\
.astype(str)

In [17]:
listings['host_listings_count_range'] = listings['host_listings_count_range'].apply(format_interval)

In [18]:
listings['host_years_active_range'] = listings['host_years_active_range'].apply(format_interval)

In [20]:
listings[[
    'host_response_rate', 
    'host_acceptance_rate'
]] = listings[[
    'host_response_rate', 
    'host_acceptance_rate'
]].fillna(0).astype(int)

In [21]:
listings['host_verifications'] = listings['host_verifications']\
.apply(ast.literal_eval)\
.apply(lambda x: ', '.join(x))

In [22]:
listings['last_review_month'] = listings['last_review'].dt.month

In [23]:
for verification in ['email', 'phone', 'work_email']:
    listings[verification] = listings['host_verifications'].str.contains(verification).astype(int)

In [24]:
listings = listings.drop(
    columns = constants.list_drop_listings
)

In [25]:
listings['sentiment'] = listings['comments']\
.apply(features.calculate_sentiment)

In [26]:
listings['host_response_time'] = listings['host_response_time'].fillna('not informed')

In [27]:
listings['host_years_active'] = listings['host_years_active'].astype(str)

In [28]:
listings['is_positive_sentiment'] = listings['sentiment']\
.apply(lambda x: 1 if x > 0.5 else 0)

In [29]:
listings.to_parquet('data/silver/listings.parquet')